In [2]:
import numpy as np
from dadk.QUBOSolverCPU import *
from dadk.BinPol import *
from datetime import datetime
import pandas as pd

# Input Data

In [3]:
# Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].values.tolist()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].values.tolist()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].values.tolist()

n_assets = len(asset_quantity)

# Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].values.tolist()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].values.tolist()

n_accounts = len(account_exposure)

# Haircuts 
haircuts = (pd.read_csv("data/sample_haircuts.csv", header=None).values.tolist())

#Single Limits
#single_limits = (pd.read_csv("data/sample_single_limits.csv", header=None).values.tolist())

# Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

## Penalty Terms

In [4]:
# Penalty terms
lambda_cost_fn = 1e2
lambda_consistency = 5e5
lambda_exposure = 1
#lambda_single_lim = 1

# Defining the QUBO

In [5]:
# n-bit binary variables
n_binary = 7
max_binary_value = 2**(n_binary) - 1 

binary_coeff_vector = np.zeros(n_binary)

for i in range(n_binary):
    binary_coeff_vector[i] = 2**i

# Binary_variables
binary_variables = VarShapeSet(BitArrayShape(name='bin_vars', shape=(n_assets, n_accounts, n_binary), axis_names=['Assets', 'Accounts', 'Binary']))



consistency_slack_var = VarShapeSet(BitArrayShape(name='consistency_slack_var', shape=(n_assets, (int(np.ceil(np.log2(max_binary_value))))), axis_names=['Assets', 'Slack']))

# Constructing the QUBO equation
This involves the QUBO objective function, the consistency contraints, the exposure constraints.

In [6]:
# QUBO objective function 
cost_function = BinPol(binary_variables)
for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            cost_function.add_term(cost_factor_matrix[i][j]*binary_coeff_vector[k]/max_binary_value,(("bin_vars", i, j, k),))

In [7]:
# Consistency constraint 
con_constraint = BinPol(consistency_slack_var)

for i in range(n_assets):
    aux_term = BinPol(binary_variables)
    consistency_penalty_slack = BinPol(consistency_slack_var)
    #slack
    for l in range((int(np.ceil(np.log2(max_binary_value))))):
        consistency_penalty_slack.add_term(2**l,(('consistency_slack_var', i,l)))
    #decision variable
    for j in range(n_accounts):
        for k in range(n_binary):
            aux_term.add_term(binary_coeff_vector[k],(("bin_vars", i, j, k),))
    aux_term.add_term(-max_binary_value,()).add(consistency_penalty_slack)
    aux_term.power(2)
    con_constraint.add(aux_term)
#pprint(con_constraint)


161290 - 504.0 x_0 + 16.0 x_0 x_1 + 32.0 x_0 x_2 + 64.0 x_0 x_3 + 128.0 x_0 x_4 + 256.0 x_0 x_5 + 512.0 x_0 x_6 + 4.0 x_0 x_7 + 8.0 x_0 x_8 + 16.0 x_0 x_9 + 32.0 x_0 x_10 + 64.0 x_0 x_11 + 128.0 x_0 x_12 + 256.0 x_0 x_13 + 4.0 x_0 x_14 + 8.0 x_0 x_15 + 16.0 x_0 x_16 + 32.0 x_0 x_17 + 64.0 x_0 x_18 + 128.0 x_0 x_19 + 256.0 x_0 x_20 + 4.0 x_0 x_21 + 8.0 x_0 x_22 + 16.0 x_0 x_23 + 32.0 x_0 x_24 + 64.0 x_0 x_25 + 128.0 x_0 x_26 + 256.0 x_0 x_27 + 4.0 x_0 x_28 + 8.0 x_0 x_29 + 16.0 x_0 x_30 + 32.0 x_0 x_31 + 64.0 x_0 x_32 + 128.0 x_0 x_33 + 256.0 x_0 x_34 - 1000.0 x_1 + 64.0 x_1 x_2 + 128.0 x_1 x_3 + 256.0 x_1 x_4 + 512.0 x_1 x_5 + 1024.0 x_1 x_6 + 8.0 x_1 x_7 + 16.0 x_1 x_8 + 32.0 x_1 x_9 + 64.0 x_1 x_10 + 128.0 x_1 x_11 + 256.0 x_1 x_12 + 512.0 x_1 x_13 + 8.0 x_1 x_14 + 16.0 x_1 x_15 + 32.0 x_1 x_16 + 64.0 x_1 x_17 + 128.0 x_1 x_18 + 256.0 x_1 x_19 + 512.0 x_1 x_20 + 8.0 x_1 x_21 + 16.0 x_1 x_22 + 32.0 x_1 x_23 + 64.0 x_1 x_24 + 128.0 x_1 x_25 + 256.0 x_1 x_26 + 512.0 x_1 x_27 + 8.0 x_1 x

In [8]:
#Exposure constraint
exposure_penalty_term = BinPol(binary_variables)

for j in range(n_accounts):
    aux_term1 = BinPol(binary_variables)
    for i in range(n_assets):
        for k in range(n_binary):
            aux_term1.add_term(binary_coeff_vector[k]*asset_quantity[i]*asset_value[i]*haircuts[i][j]/(100*max_binary_value),(("bin_vars", i, j, k),))
    aux_term1.add_term(-account_exposure[j],())
    aux_term1.power(2)
    exposure_penalty_term.add(aux_term1)

#pprint(exposure_penalty_term)

1654878553038.0305 - 4373509416.234929 x_0 + 28262320.462951034 x_0 x_1 + 56524640.92590207 x_0 x_2 + 113049281.85180414 x_0 x_3 + 226098563.70360827 x_0 x_4 + 452197127.40721655 x_0 x_5 + 904394254.8144331 x_0 x_6 + 59308868.13317204 x_0 x_35 + 118617736.26634409 x_0 x_36 + 237235472.53268817 x_0 x_37 + 474470945.06537634 x_0 x_38 + 948941890.1307527 x_0 x_39 + 1897883780.2615054 x_0 x_40 + 3795767560.5230107 x_0 x_41 + 10137736.043829752 x_0 x_70 + 20275472.087659504 x_0 x_71 + 40550944.17531901 x_0 x_72 + 81101888.35063802 x_0 x_73 + 162203776.70127603 x_0 x_74 + 324407553.40255207 x_0 x_75 + 648815106.8051041 x_0 x_76 + 22217290.490032703 x_0 x_105 + 44434580.980065405 x_0 x_106 + 88869161.96013081 x_0 x_107 + 177738323.92026162 x_0 x_108 + 355476647.84052324 x_0 x_109 + 710953295.6810465 x_0 x_110 + 1421906591.362093 x_0 x_111 + 80589268.84884264 x_0 x_140 + 161178537.69768527 x_0 x_141 + 322357075.39537054 x_0 x_142 + 644714150.7907411 x_0 x_143 + 1289428301.5814822 x_0 x_144 + 2

In [ ]:
# # Single limit constraint

# #slack part

# single_limit_penalty_slack = []
# for i in range(n_assets):
#     aux_array = []
#     for j in range(n_accounts):
#         aux_term = BinPol(var_shape_set=single_lim_slack_variables)
#         for l in range(int(num_slack_single_limit_arr[i][j])):
#             aux_term.add_term(2**l,((f"singlelim_slack_asset{i+1}_account{j+1}",0,l),))
#         aux_array.append(aux_term)
#     single_limit_penalty_slack.append(aux_array)

# #decision variable part

# single_limit_penalty_term = BinPol(binary_variables)
# #This for loop works in conjuction with the nontraditional way of creating slack variables
# for i in range(n_assets):
#     for j in range(n_accounts):
#         aux_term1 = BinPol(binary_variables)
#         for k in range(n_binary):
#             aux_term1.add_term(binary_coeff_vector[k],(('bin_vars',i,j,k),))
#         aux_term1.add_term(-single_limits[i][j]*max_binary_value/asset_quantity[i])
#         aux_term1.add(single_limit_penalty_slack[i][j])
#         single_limit_penalty_term.add(aux_term1**2)

In [9]:
# Final QUBO
QUBO_equation = lambda_cost_fn*cost_function + lambda_consistency*con_constraint  + lambda_exposure*exposure_penalty_term #+lambda_single_lim*single_limit_penalty_term

# Run Solver

In [10]:
# Define solver parameters
solver = QUBOSolverCPU(
   
    number_iterations    = 1000,               # total number of itrations per run
    number_runs          = 1000,                 # number of stochastically independant runs
    temperature_start    = 5000,               # start temperature for annealing as float value
    temperature_end      = 10,                 # end temperature for annealing as float value 
    temperature_mode     = 0,                         # 0: reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                                      # 1: reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                                      # 2: reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    temperature_decay    = 0.0095,             # see temperature_mode 0
    temperature_interval = 1,                  # see temperature_mode 0
    offset_increase_rate = 5000.0,             # increase of dynamic offset when no bit selected, set to 0.0 to switch off dynamic offset
    graphics             = True                # create data for graphics output
)

In [11]:
# Run Solver
start=datetime.now()
solution_list = solver.minimize(QUBO_equation)

#Get results
solution = solution_list.get_minimum_energy_solution()
configuration = solution.configuration
my_bit_array = solution.extract_bit_array("bin_vars")

# Verify Results

In [12]:
bitstring = ''

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=(2**k)*my_bit_array[i][j][k]
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]/100
    allocation_solution[i] = asset
    

=====Each Individual Allocation Percentage=====
Assign 0.00% of asset 1 to account 1
Assign 50.39% of asset 1 to account 2
Assign 0.00% of asset 1 to account 3
Assign 0.00% of asset 1 to account 4
Assign 56.69% of asset 1 to account 5
Assign 0.00% of asset 2 to account 1
Assign 1.57% of asset 2 to account 2
Assign 25.20% of asset 2 to account 3
Assign 0.00% of asset 2 to account 4
Assign 0.00% of asset 2 to account 5
Assign 5.51% of asset 3 to account 1
Assign 5.51% of asset 3 to account 2
Assign 55.91% of asset 3 to account 3
Assign 2.36% of asset 3 to account 4
Assign 3.94% of asset 3 to account 5
Assign 25.20% of asset 4 to account 1
Assign 21.26% of asset 4 to account 2
Assign 18.90% of asset 4 to account 3
Assign 0.00% of asset 4 to account 4
Assign 0.00% of asset 4 to account 5
Assign 0.00% of asset 5 to account 1
Assign 0.00% of asset 5 to account 2
Assign 0.00% of asset 5 to account 3
Assign 3.15% of asset 5 to account 4
Assign 4.72% of asset 5 to account 5
Assign 3.15% of asse

In [13]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

====Validation for Consistency====
107.09% of asset 1 posted
26.77% of asset 2 posted
73.23% of asset 3 posted
65.35% of asset 4 posted
7.87% of asset 5 posted
15.75% of asset 6 posted
100.79% of asset 7 posted
53.54% of asset 8 posted
87.40% of asset 9 posted
92.13% of asset 10 posted


In [14]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]:,} required")

=====Validation for Exposure======
$830,366.3175744611 of collateral posted, $824000.0 required
$675,643.9182669547 of collateral posted, $669292.0 required
$554,532.0712548938 of collateral posted, $548600.0 required
$71,420.97039157938 of collateral posted, $64844.52 required
$479,563.459628038 of collateral posted, $472000.0 required


In [ ]:
#print("===Validation of Single limits =====")
# for i in range(n_assets):
#     for j in range(n_accounts):
#         decimal = 0
#         for k in range(n_binary):
#             decimal +=(2**k)*my_bit_array[i][j][k]
#         allocation = (decimal/max_binary_value)*asset_quantity[i] 
#         print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
#         print(f"Constraint Satisified?: {allocation <= single_limits[i][j]}")
        

In [18]:
print(f"The objective function is: {cost_function.compute(configuration)}")
print("The optimal objective function is: 0.4299816571359823")

The objective function is: 4.118110236220471
The optimal objective function is: 0.4299816571359823


In [16]:
# Print runtime
print(datetime.now()-start)

0:10:49.711807


# Export to .csv

In [17]:
# Export to .csv
Q_value_balanced_fujitsu = np.zeros(shape=(n_assets,n_accounts))


for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=binary_coeff_vector[k]*my_bit_array[i][j][k]
        asset += decimal*(100/max_binary_value)
        Q_value_balanced_fujitsu[i][j] = decimal/max_binary_value

print(Q_value_balanced_fujitsu)

#Uncomment the line below to save the Data to csv

# pd.DataFrame(Q_value_balanced_fujitsu).to_csv("Q_value_balanced_fujitsu_no_single_lim_trial.csv", header=None, index=None)

[[0.         0.50393701 0.         0.         0.56692913]
 [0.         0.01574803 0.2519685  0.         0.        ]
 [0.05511811 0.05511811 0.55905512 0.02362205 0.03937008]
 [0.2519685  0.21259843 0.18897638 0.         0.        ]
 [0.         0.         0.         0.03149606 0.04724409]
 [0.03149606 0.06299213 0.         0.         0.06299213]
 [0.50393701 0.50393701 0.         0.         0.        ]
 [0.53543307 0.         0.         0.         0.        ]
 [0.02362205 0.71653543 0.00787402 0.         0.12598425]
 [0.23622047 0.49606299 0.02362205 0.02362205 0.14173228]]
